In [101]:
include("../code/headers.jl")

In [204]:
n = 1000
m = 400
maxiter=Int(1e2)
tol=1e-13

1.0e-13

In [168]:
H = [1 1 1]
m,n = size(H)

(1, 3)

In [218]:
fg = ldpc_graphGF2(n,m)
# fg = FactorGraphGF2(H)
for f in 1:m
   fg.mfv[f] .= zeros(length(fg.mfv[f])) 
end
s = [-1, 1, 1] #+ 1e-8*randn(n)
s = 1e-8*randn(n)
fg.fields .= copy(s);

In [222]:
f = 1
fg.fields[fg.Fneigs[f]]

5-element Array{Float64,1}:
 -3.564735628744456e-9
 -5.187591079027455e-9
  4.4513305886594555e-9
 -1.03951946058323e-8
 -9.873158842351336e-10

In [219]:
t = Prod{Float64}()
for (i, v) in enumerate(fg.Fneigs[f])
    # Avoid Inf-Inf=NaN
    if fg.fields[v] == fg.mfv[f][i] 
        fg.fields[v] = 0.0
    else
        fg.fields[v] -= fg.mfv[f][i]
    end
    t *= tanh(beta*fg.fields[v])
end
t

Prod{Float64}(8.448345361356546e-43, 0, 0)

In [221]:
for (i, v) in enumerate(fg.Fneigs[f])
    m = 1/beta*atanh(t/tanh(beta*fg.fields[v]))
    isnan(m) && @show t,tanh(beta*fg.fields[v])
    fg.mfv[f][i] = m
    # Update belief after updating the message
    isnan(fg.fields[v]+m) && (@show f,v,fg.fields[v],m; error("NaN in field"))
    fg.fields[v] += m
end
fg.fields[fg.Fneigs[f]]

5-element Array{Float64,1}:
 -3.564735628744456e-9
 -5.187591079027455e-9
  4.4513305886594555e-9
 -1.03951946058323e-8
 -9.873158842351336e-10

In [216]:
ϵ = 0.0
@showprogress for it in 1:1
    ϵ = onebpiter!(fg, BP(), beta=0.1)
end
σ = sign.(fg.fields)
@assert all(σ .!= 0)
x = σ .== -1
nunsat = sum(fg.H*x .% 2)
nunsat, ϵ

(216, 5.017931275567142e-26)

In [217]:
fg.fields - s

1000-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [197]:
fg.mfv

400-element Array{Array{Float64,1},1}:
 [8.679291836066606e-36, -1.3981647496035353e-37, -9.371623910495656e-35, -7.042967684311574e-37, -2.928891069057608e-36]
 [-2.8645412866120282e-33, 1.5512306651594406e-33, 1.832079818702644e-33, 5.870264179678719e-32, -3.743732527107919e-33]
 [3.0673065858763407e-34, -4.695092854207344e-34, 3.4210717888666774e-34, 8.784644562443347e-34, -3.3765610990853887e-34]
 [-8.691354630678946e-33, 5.132366761953097e-34, -2.99852181718419e-33, 1.20052750409494e-33, 1.5071531613208639e-33]
 [-4.855898662613959e-34, -6.2711781089633986e-34, 6.520146007914764e-34, 8.621055065206277e-34, 1.179323387697823e-33]
 [2.7263098784342006e-33, -3.36155169886522e-33, -2.409269547098881e-33, -5.6052403339833424e-33, -2.500835568554083e-33]
 [5.291799916785667e-35, -8.341770063927222e-35, 3.1773883482183516e-35, -3.710481606277312e-35, 2.861738668923873e-34]
 [5.880477031293699e-33, -3.217241829131818e-33, -2.7062269752794197e-32, -2.784829298923657e-33, -2.410489066432453

In [60]:
σ = sign.(fg.fields)
@assert all(σ .!= 0)
x = σ .== -1
nunsat = sum(fg.H*x .% 2)

0